# Data

In [3]:
import json
with open('./data/hormozi_tweets.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

# make a list of 300 tweets
tweets = [item['tweet'] for item in data][:300]

# Baseline Tweet Generator

In [3]:
import random
import openai
import os
from dotenv import load_dotenv

load_dotenv()

random_tweets = random.sample(tweets, 10)

prompt = f"""
Past Tweets:
{random_tweets}

Create a new tweet based on the past tweets.
"""

openai.api_key = os.environ.get('OPENAI_API_KEY')

def generate_tweet(prompt):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that creates tweets similar to past tweets from the user."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=280,
    )
    return response.choices[0].message.content.strip().lower()

# Create a new tweet
new_tweet = generate_tweet(prompt)

In [4]:
new_tweet

'"success isn\'t about never hitting a low point; it\'s about recognizing that those moments are opportunities to push through while others quit. remember, every setback is just a setup for your comeback. 💪"'

# First experiment

DSPy recommends that you start with the simplest solution and add complexity, so that is what we'll do. For our first experiement, we'll just use a ChainOfThought module to generate tweets based on all the tweets we've previously seen.

In [14]:
import json
with open('./data/hormozi_tweets.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

# make a list of 300 tweets
tweets = [{'tweet': item['tweet'], 'engagement': item['replies'] + item['retweets'] +item['likes']} for item in data][:300]

In [16]:
tweets[2]

{'tweet': 'Rush is an illusion.', 'engagement': 908}

In [13]:
import dspy
from dspy.primitives import Example
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot


In [20]:
tweets

[{'tweet': 'Stop “friend financing”', 'engagement': 137},
 {'tweet': 'A message for friends of entrepreneurs:\n\nDon’t buy them a present. \n\nBuy their product and leave a nice review or tell them why it wasn’t good enough to deserve it (an even better gift).\n\nNo one needs more stuff. \nEveryone could use more support.',
  'engagement': 979},
 {'tweet': 'Rush is an illusion.', 'engagement': 908},
 {'tweet': 'Advice to strong men:\n\nFind a strong woman.',
  'engagement': 3574},
 {'tweet': 'Reminder:\n\nDeath tax is 100% for everyone.\n\n(Because whatever you have isn’t yours anymore after you die).',
  'engagement': 1136},
 {'tweet': 'When it comes to talent, “good enough” often isn’t.',
  'engagement': 941},
 {'tweet': 'Squeeze all the potential you’ve got into reality.\n\nEvery. Last. Drop.',
  'engagement': 3391},
 {'tweet': 'It’s not about doing your best, it’s about doing what’s required.\n\nAnd sometimes that means your best just needs to get better.',
  'engagement': 2555},
 

## Annotating tweets
Our tweets don't have any topic or label associated with them that might help us optimize and generate more tweets. We can annotate them by getting an LLM to find out the topic they fall under.

In [21]:
import dspy

# Convert tweets to Example objects
dataset = [dspy.Example(tweet=tweet).with_inputs("tweet", "engagement") for tweet in tweets]

In [22]:
class TopicPredictor(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("tweet -> topic")

    def forward(self, tweet):
        return self.prog(tweet=tweet)

In [26]:
mini = dspy.LM(model='gpt-4o')
dspy.configure(lm = mini)

In [27]:
# Initialize the topic predictor
topic_predictor = TopicPredictor()

# Annotate each tweet with a predicted topic
annotated_dataset = []
for example in dataset:
    response = topic_predictor(example.tweet)
    annotated_example = example.copy()
    annotated_example.topic = response.topic
    annotated_dataset.append(annotated_example)

# Print annotated dataset
for example in annotated_dataset:
    print(f"Tweet: {example.tweet}, Topic: {example.topic}")

Tweet: {'tweet': 'Stop “friend financing”', 'engagement': 137}, Topic: Personal Finance
Tweet: {'tweet': 'A message for friends of entrepreneurs:\n\nDon’t buy them a present. \n\nBuy their product and leave a nice review or tell them why it wasn’t good enough to deserve it (an even better gift).\n\nNo one needs more stuff. \nEveryone could use more support.', 'engagement': 979}, Topic: Support for Entrepreneurs
Tweet: {'tweet': 'Rush is an illusion.', 'engagement': 908}, Topic: Philosophy
Tweet: {'tweet': 'Advice to strong men:\n\nFind a strong woman.', 'engagement': 3574}, Topic: Relationships
Tweet: {'tweet': 'Reminder:\n\nDeath tax is 100% for everyone.\n\n(Because whatever you have isn’t yours anymore after you die).', 'engagement': 1136}, Topic: Philosophy, Mortality
Tweet: {'tweet': 'When it comes to talent, “good enough” often isn’t.', 'engagement': 941}, Topic: Talent and Excellence
Tweet: {'tweet': 'Squeeze all the potential you’ve got into reality.\n\nEvery. Last. Drop.', 'en

In [28]:
annotated_dataset[0]

Example({'tweet': {'tweet': 'Stop “friend financing”', 'engagement': 137}, 'topic': 'Personal Finance'}) (input_keys=None)

We now have our annotated data. We can save it to use later and train a tweet writer.

In [30]:
class TweetWriter(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("topic -> tweet")

    def forward(self, topic):
        return self.prog(topic=topic)

In [68]:
class Assess(dspy.Signature):
    """Assess the creativity of a tweet."""
    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="Score between 1 and 5")

def creativity_metric(gold, pred, trace=None):
    tweet = pred.tweet
    creativity_question = "Rate the creativity of the following tweet on a scale of 1 to 5."
    creativity_score = dspy.Predict(Assess)(
            assessed_text=tweet,
            assessment_question=creativity_question
        )
    print(creativity_score)
    score = int(creativity_score.assessment_answer.strip())
    return score

In [32]:
uncompiled_tweet_writer = TweetWriter()

In [33]:
uncompiled_tweet_writer("Business")

Prediction(
    reasoning='Discussing business topics on social media can provide valuable insights, foster networking opportunities, and keep followers informed about industry trends and best practices. A well-crafted tweet can engage your audience and position you as a thought leader in your field.',
    tweet='In the ever-evolving world of business, staying ahead means embracing innovation and continuous learning. 📈💡 What new strategies are you implementing to drive growth in 2023? #BusinessGrowth #Innovation #Leadership'
)

Let's evaluate our uncompiled tweet writer

In [72]:
flattened_annotated_dataset = [
    dspy.Example(tweet=item.tweet["tweet"], engagement = item.tweet["engagement"], topic=item["topic"]).with_inputs("topic")
    for item in annotated_dataset
]

In [45]:
import pickle

with open('data/annotated_dataset.pkl', 'wb') as f:
    pickle.dump(flattened_annotated_dataset, f) 

In [52]:
dev_set_n = 250
trainset = flattened_annotated_dataset[:dev_set_n]
devset = flattened_annotated_dataset[dev_set_n:]

In [91]:
evaluator = Evaluate(devset=devset[:10], metric=creativity_metric, num_threads=4, display_progress=True, display_table=5, provide_traceback=True)
evaluator(uncompiled_tweet_writer) 

Prediction(
    assessment_answer='4'
)
Prediction(
    assessment_answer='4'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='2'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='4'
)
Prediction(
    assessment_answer='3'
)


Average Metric: 32 / 10  (320.0): 100%|██████████| 10/10 [00:00<00:00, 192.90it/s]


,example_tweet,engagement,topic,reasoning,pred_tweet,creativity_metric
0,"Doing a thing increases your ability to learn about it. In order words, do the thing, then read about it. The experience will give you...",3388,Learning and Experience,"Learning and experience are deeply interconnected. Learning provides the theoretical foundation and knowledge base, while experience offers practical application and real-world insights. Together, they create...","Learning gives you the knowledge, but experience turns that knowledge into wisdom. Embrace both for a well-rounded journey of growth! 🌱✨ #Learning #Experience #Growth",✔️ [4]
1,Anyone else ever accidentally get in the shower with a hat still on? https://t.co/t9IOAGcC1h,1647,Everyday Life/Humor,"Humor is a great way to connect with people on social media, especially when it relates to everyday life. It makes the content relatable and...","Why is it that the moment you decide to eat healthy, your fridge suddenly looks like a junk food paradise? 🍕🍫 #EverydayStruggles #HealthyEating",✔️ [4]
2,"If you have all the information to make a perfect decision, you missed the opportunity.",4534,Decision Making,"Decision making is a critical skill that impacts every aspect of our lives, from personal choices to professional strategies. Effective decision making involves evaluating options,...","Good decisions come from experience, and experience comes from bad decisions. 🧠✨ Master the art of decision making by evaluating your options, considering potential outcomes,...",✔️ [3]
3,"I’m bad at starting new habits. I’m also pretty bad at keeping habits. In the last ten years, the only small habits I’ve been able...",3551,Habit Formation,Habit formation is a crucial aspect of personal development and productivity. Understanding how habits are formed and maintained can help individuals make positive changes in...,"Building good habits is the key to unlocking your potential! Start small, stay consistent, and watch how tiny changes lead to big results. 💪 #HabitFormation...",✔️ [3]
4,The reason you are stressed is you have decisions to make and you’re not making them.,9078,Mental Health,Mental health is a crucial aspect of overall well-being that often gets overlooked. Raising awareness and encouraging open conversations can help reduce stigma and provide...,Your mental health matters just as much as your physical health. Don't hesitate to reach out for support if you need it. Let's break the...,✔️ [3]


320.0

### For unoptimized tweet writer, our score on the devset is: 320

# BootstrapFewShot

In [89]:
config = dict(max_bootstrapped_demos=25, max_labeled_demos=4)
teleprompter = BootstrapFewShot(metric=creativity_metric, **config)
optimized_tweet_writer = teleprompter.compile(TweetWriter(), trainset=trainset[:10])

Prediction(
    assessment_answer='3'
)


Prediction(
    assessment_answer='3'
)


Prediction(
    assessment_answer='4'
)


Prediction(
    assessment_answer='3'
)


Prediction(
    assessment_answer='3'
)


...

Prediction(
    assessment_answer='3'
)


Prediction(
    assessment_answer='3'
)


Prediction(
    assessment_answer='3'
)


100%|██████████| 10/10 [00:20<00:00,  2.09s/it]

Prediction(
    assessment_answer='3'
)
Bootstrapped 10 full traces after 10 examples in round 0.


In [90]:
dspy.settings.configure(backoff_time=5)
evaluator = Evaluate(devset=devset[:10], metric=creativity_metric, num_threads=2, display_progress=True, display_table=5, provide_traceback=True)
evaluator(optimized_tweet_writer)

Average Metric: 4 / 1  (400.0):  10%|█         | 1/10 [00:01<00:16,  1.85s/it]

Prediction(
    assessment_answer='4'
)


Average Metric: 7 / 2  (350.0):  20%|██        | 2/10 [00:02<00:09,  1.16s/it]

Prediction(
    assessment_answer='3'
)


Average Metric: 10 / 3  (333.3):  30%|███       | 3/10 [00:03<00:07,  1.13s/it]

...

Prediction(
    assessment_answer='4'
)


Average Metric: 32 / 10  (320.0): 100%|██████████| 10/10 [00:10<00:00,  1.05s/it]

Prediction(
    assessment_answer='3'
)


,example_tweet,engagement,topic,reasoning,pred_tweet,creativity_metric
0,"Doing a thing increases your ability to learn about it. In order words, do the thing, then read about it. The experience will give you...",3388,Learning and Experience,Not supplied for this particular example.,"Learning gives you knowledge, but experience gives you wisdom. Embrace both, and you'll navigate life with confidence and insight. 📚🌍 #Learning #Experience",✔️ [3]
1,Anyone else ever accidentally get in the shower with a hat still on? https://t.co/t9IOAGcC1h,1647,Everyday Life/Humor,Not supplied for this particular example.,"Adulting is like folding a fitted sheet. No one really knows how to do it, but we all pretend we do. 😂 #EverydayLife #Humor",✔️ [4]
2,"If you have all the information to make a perfect decision, you missed the opportunity.",4534,Decision Making,Not supplied for this particular example.,"Every decision shapes your future. Trust your instincts, gather the facts, and don't be afraid to take calculated risks. Growth often lies just outside your...",✔️ [3]
3,"I’m bad at starting new habits. I’m also pretty bad at keeping habits. In the last ten years, the only small habits I’ve been able...",3551,Habit Formation,Not supplied for this particular example.,"Small, consistent actions lead to big changes. Focus on building one positive habit at a time, and watch how they transform your life. 🌟 #HabitFormation...",✔️ [3]
4,The reason you are stressed is you have decisions to make and you’re not making them.,9078,Mental Health,Not supplied for this particular example.,"Taking care of your mental health is just as important as taking care of your physical health. Reach out, talk about it, and seek help...",✔️ [3]


320.0

It seems like BootstrapFewShots isn't giving us the results we want since it didn't improve on an uncompiled program. Let's try a few more examples and BootstrapFewShotWithRandomSearch

# BootstrapFewShotWithRandomSearch

DSPy suggests that you use BootstrapFewShotWithRandomSearch for upto 50 examples.

In [101]:
from dotenv import load_dotenv

load_dotenv()

True

In [108]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

dspy.settings.configure(backoff_time=10)
config = dict(max_labeled_demos = 1, max_bootstrapped_demos=1, num_candidate_programs=2)
teleprompter = BootstrapFewShotWithRandomSearch(metric = creativity_metric, **config)

rs_optimized_tweet_writer = teleprompter.compile(TweetWriter(), trainset=trainset[:20])

Going to sample between 1 and 1 traces per predictor.
Will attempt to bootstrap 2 candidate sets.
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='4'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='4'
)
Prediction(
    assessment_answer='4'
)
Prediction(
    assessment_answer='2'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='2'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)


Average Metric: 61 / 20  (305.0): 100%|██████████| 20/20 [00:00<00:00, 415.02it/s] 


New best score: 305.0 for seed -3
Scores so far: [305.0]
Best score so far: 305.0


Average Metric: 9 / 3  (300.0):  10%|█         | 2/20 [00:02<00:18,  1.01s/it]

Prediction(
    assessment_answer='4'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='2'
)


...

Prediction(
    assessment_answer='2'
)


Average Metric: 59 / 19  (310.5):  95%|█████████▌| 19/20 [00:24<00:01,  1.75s/it]

Prediction(
    assessment_answer='3'
)


Average Metric: 62 / 20  (310.0): 100%|██████████| 20/20 [00:24<00:00,  1.24s/it]

Prediction(
    assessment_answer='3'
)
Scores so far: [305.0, 300.0, 310.0, 305.0, 310.0]
Best score so far: 310.0
5 candidate programs found.


In [110]:
dspy.settings.configure(backoff_time=5)
evaluator = Evaluate(devset=devset[:10], metric=creativity_metric, num_threads=2, display_progress=True, display_table=5, provide_traceback=True)
evaluator(rs_optimized_tweet_writer)

Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='4'
)
Prediction(
    assessment_answer='2'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='2'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)


Average Metric: 29 / 10  (290.0): 100%|██████████| 10/10 [00:00<00:00, 1355.89it/s]


,example_tweet,engagement,topic,reasoning,pred_tweet,creativity_metric
0,"Doing a thing increases your ability to learn about it. In order words, do the thing, then read about it. The experience will give you...",3388,Learning and Experience,"Learning and experience are fundamental to personal and professional growth. They enable us to acquire new skills, adapt to changing environments, and make informed decisions....",Never stop learning and seeking new experiences! They are the building blocks of growth and success. Embrace every opportunity to expand your horizons. 🌟📚 #LifelongLearning...,✔️ [3]
1,Anyone else ever accidentally get in the shower with a hat still on? https://t.co/t9IOAGcC1h,1647,Everyday Life/Humor,Humor in everyday life helps to lighten the mood and make daily routines more enjoyable. Sharing funny observations or experiences can bring a smile to...,Why do we always find the one thing we lost right after we buy a replacement? It's like the universe is playing hide and seek...,✔️ [4]
2,"If you have all the information to make a perfect decision, you missed the opportunity.",4534,Decision Making,"Decision making is a fundamental skill that impacts every aspect of our lives, from personal choices to professional paths. Effective decision making involves evaluating options,...","Good decision making is the cornerstone of success! Take the time to weigh your options, consider the outcomes, and choose wisely. Your future depends on...",✔️ [2]
3,"I’m bad at starting new habits. I’m also pretty bad at keeping habits. In the last ten years, the only small habits I’ve been able...",3551,Habit Formation,"Habit formation is essential for personal growth and achieving long-term goals. By developing positive habits, we can improve our productivity, health, and overall well-being. Understanding...","Building good habits is the foundation of success! Start small, stay consistent, and watch your life transform. 🌱🔄 #HabitFormation #PersonalGrowth",✔️ [3]
4,The reason you are stressed is you have decisions to make and you’re not making them.,9078,Mental Health,"Mental health is a vital component of overall well-being, impacting how we think, feel, and act. Addressing mental health issues and promoting mental wellness can...","Your mental health matters just as much as your physical health. Take time to care for your mind, seek support when needed, and remember that...",✔️ [3]


290.0

Let's try MIPROv2 and see if we can improve on this, then we could try out maybe changing the metric to better optimize our writer.

In [111]:
optimized_tweet_writer.save(path="fewshot")

[('prog', Predict(StringSignature(topic -> reasoning, tweet
    instructions='Given the fields `topic`, produce the fields `tweet`.'
    topic = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Topic:', 'desc': '${topic}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    tweet = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Tweet:', 'desc': '${tweet}'})
)))]


In [112]:
rs_optimized_tweet_writer.save(path="fewshotwithrs")

[('prog', Predict(StringSignature(topic -> reasoning, tweet
    instructions='Given the fields `topic`, produce the fields `tweet`.'
    topic = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Topic:', 'desc': '${topic}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    tweet = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Tweet:', 'desc': '${tweet}'})
)))]


# MIPROv2

In [114]:
# Import the optimizer
from dspy.teleprompt import MIPROv2

# Initialize optimizer
teleprompter = MIPROv2(
    metric=creativity_metric,
    num_candidates=7,
    init_temperature=0.5,
    verbose=False,
    num_threads=2,
)

mipro_optimized_program = teleprompter.compile(
    TweetWriter(),
    trainset=trainset,
    max_bootstrapped_demos=3,
    max_labeled_demos=4,
    num_trials=10,
    minibatch_size=25,
    minibatch_full_eval_steps=10,
    minibatch=True, 
)

# Save optimize program for future use
mipro_optimized_program.save("mipro_optimized")

Projected Language Model (LM) Calls

Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 7 * 1 lm calls in program + (2) lm calls in program aware proposer = 19 prompt model calls
- Task Model: 25 examples in minibatch * 10 batches + 200 examples in val set * 1 full evals = 300 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your

  2%|▏         | 1/50 [00:02<01:55,  2.37s/it]

Prediction(
    assessment_answer='2'
)


  4%|▍         | 2/50 [00:04<01:53,  2.36s/it]

Prediction(
    assessment_answer='3'
)


...

Prediction(
    assessment_answer='3'
)
Prediction(
    assessment_answer='3'
)


Average Metric: 619 / 199  (311.1): 100%|█████████▉| 199/200 [10:29<00:02,  2.89s/it]

Prediction(
    assessment_answer='3'
)


Average Metric: 622 / 200  (311.0): 100%|██████████| 200/200 [10:33<00:00,  3.17s/it]

Prediction(
    assessment_answer='3'
)
Best full eval score so far! Score: 311.0


[('prog', Predict(StringSignature(topic -> reasoning, tweet
    instructions='Given the fields `topic`, produce the fields `tweet`.'
    topic = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Topic:', 'desc': '${topic}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    tweet = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Tweet:', 'desc': '${tweet}'})
)))]


In [115]:
evaluator = Evaluate(devset=devset[:10], metric=creativity_metric, num_threads=2, display_progress=True, display_table=5, provide_traceback=True)
evaluator(mipro_optimized_program)

Average Metric: 4 / 1  (400.0):  10%|█         | 1/10 [00:08<01:14,  8.31s/it]

Prediction(
    assessment_answer='4'
)


Average Metric: 8 / 2  (400.0):  20%|██        | 2/10 [00:09<00:30,  3.83s/it]

Prediction(
    assessment_answer='4'
)


Average Metric: 20 / 6  (333.3):  50%|█████     | 5/10 [00:18<00:19,  3.96s/it]

...

Prediction(
    assessment_answer='4'
)


Average Metric: 33 / 10  (330.0): 100%|██████████| 10/10 [00:43<00:00,  4.39s/it]

Prediction(
    assessment_answer='3'
)


,example_tweet,engagement,topic,reasoning,pred_tweet,creativity_metric
0,"Doing a thing increases your ability to learn about it. In order words, do the thing, then read about it. The experience will give you...",3388,Learning and Experience,"Learning and experience are two sides of the same coin. While learning provides the theoretical foundation and knowledge, experience offers practical application and deeper understanding....","Learning gives you the map, but experience is the journey. 🌍📚 Embrace both to navigate life’s challenges and grow continuously. #LifelongLearning #ExperienceMatters",✔️ [4]
1,Anyone else ever accidentally get in the shower with a hat still on? https://t.co/t9IOAGcC1h,1647,Everyday Life/Humor,"Everyday life is filled with small, relatable moments that can be both humorous and insightful. By highlighting these moments, we can connect with others on...","Why is it that the moment you decide to eat healthy, your fridge suddenly looks like a junk food paradise? 🍕🍫 #EverydayLife #Humor #Relatable",✔️ [4]
2,"If you have all the information to make a perfect decision, you missed the opportunity.",4534,Decision Making,"Decision making is a critical skill that impacts every aspect of our lives, from personal relationships to professional success. Effective decision making involves gathering relevant...","Struggling with decisions? Remember: indecision is a decision too. Gather info, weigh pros & cons, and align with your goals. Make choices that move you...",✔️ [3]
3,"I’m bad at starting new habits. I’m also pretty bad at keeping habits. In the last ten years, the only small habits I’ve been able...",3551,Habit Formation,"Habit formation is a crucial aspect of personal development because it allows individuals to automate positive behaviors, making them a natural part of daily life....","Want to build a new habit? Start small. Identify a cue, establish a routine, and reward yourself. Consistency is key! 🌱 #HabitFormation #PersonalGrowth",✔️ [3]
4,The reason you are stressed is you have decisions to make and you’re not making them.,9078,Mental Health,"Mental health is a crucial aspect of overall well-being, yet it is often overlooked or stigmatized. Addressing mental health involves recognizing the signs of mental...","Your mental health is just as important as your physical health. 🌟 Don't hesitate to seek help, talk openly, and support each other. Let's break...",✔️ [3]


330.0

# MIPROv2 Results: 330

MIPROv2 did increase our score from 320 -> 330.

But as we can see, most of the scores for creativity turn out to be 3 - this is becoming a problem. The LLM judge is probably having a problem defining 'creativity', especially with topics as 'dull' as business. We need a multitude of 'yes/no' metrics and we need to ensemble them to get a good sense. Plus, from the examples in the table above it seems like the tweets generated are really long. We need a conciseness metric as well - or in extreme cases, an assertion that the tweet generated be within 280 characters. It's possible the LLM has issues because we're generating + judging with the same LM, whereas the way it's generally done is a more capable model judging the generation of a smaller/less capable model. Let's try a few changes:

1) Changing the metric to encompass more measures of a tweet's 'quality'
2) using 4o-mini for generation and 4o for judging
3) Playing around with the hyperparameters of DSPy optimizers a bit more
4) Manually checking a few examples of generation.
5) Using a DSPy program that we compile to judge the quality of tweets as a metric. i.e., something like second-order optimization.
6) Using the tweets dataset for RAG since we can't train on the whole thing. (due to time constraints, Rate limit, etc.)
7) Trying a different form of annotation/labels than just topics
8) Dividing the devset into val and test sets to keep the test set for final testing and preventing data leakage

# Experiment 2: without RAG

In [116]:
flattened_annotated_dataset[:5]

[Example({'tweet': 'Stop “friend financing”', 'engagement': 137, 'topic': 'Personal Finance'}) (input_keys={'topic'}),
 Example({'tweet': 'A message for friends of entrepreneurs:\n\nDon’t buy them a present. \n\nBuy their product and leave a nice review or tell them why it wasn’t good enough to deserve it (an even better gift).\n\nNo one needs more stuff. \nEveryone could use more support.', 'engagement': 979, 'topic': 'Support for Entrepreneurs'}) (input_keys={'topic'}),
 Example({'tweet': 'Rush is an illusion.', 'engagement': 908, 'topic': 'Philosophy'}) (input_keys={'topic'}),
 Example({'tweet': 'Advice to strong men:\n\nFind a strong woman.', 'engagement': 3574, 'topic': 'Relationships'}) (input_keys={'topic'}),
 Example({'tweet': 'Reminder:\n\nDeath tax is 100% for everyone.\n\n(Because whatever you have isn’t yours anymore after you die).', 'engagement': 1136, 'topic': 'Philosophy, Mortality'}) (input_keys={'topic'})]

In [119]:
dev_set_n = 250
# Tell DSPy that the 'topic' field is the input. Any other fields are labels and/or metadata.
trainset = [x.without('engagement').with_inputs('topic') for x in flattened_annotated_dataset[:dev_set_n]]
devset = [x.without('engagement').with_inputs('topic') for x in flattened_annotated_dataset[dev_set_n:]]

In [120]:
trainset[0]

Example({'tweet': 'Stop “friend financing”', 'topic': 'Personal Finance'}) (input_keys={'topic'})

In [124]:
gpt4o = dspy.LM(model = 'gpt-4o', max_tokens=1000, model_type='chat')
mini = dspy.LM(model = 'gpt-4o-mini', max_tokens=1000, model_type='chat')
dspy.configure(lm = mini)

In [125]:
# Define the signature for automatic assessments.
class Assess(dspy.Signature):
    """Assess the quality of a tweet along the specified dimension."""

    context = dspy.InputField(desc='ignore if N/A')
    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="Yes or No")

def tweet_ensemble_metric(gold, pred, trace=None):
    topic, tweet = gold.topic, pred.tweet

    engaging = "Does the assessed text make for a self-contained, engaging tweet?"
    concise = "Does the assessed text make for a concise, cogent tweet?"
    creative = "Does the assessed text make for a creative tweet?"
    relevant = f"The text above should be relevant `{topic}`. The gold answer is `{tweet}`."
    relevant = f"{relevant} Does the assessed text above contain the gold answer?"
    
    with dspy.context(lm=gpt4o):
        relevant =  dspy.Predict(Assess)(context='N/A', assessed_text=tweet, assessment_question=relevant)
        engaging = dspy.Predict(Assess)(context='N/A', assessed_text=tweet, assessment_question=engaging)
        creative = dspy.Predict(Assess)(context='N/A', assessed_text=tweet, assessment_question=creative)
        concise = dspy.Predict(Assess)(context='N/A', assessed_text=tweet, assessment_question=concise)

    relevant, engaging, creative, concise = (m.assessment_answer.split()[0].lower() == 'yes' for m in [relevant, engaging, creative, concise])
    #takes care of length
    score = (relevant + engaging + creative + concise) if (len(tweet) <= 280) else 0

    if trace is not None: 
        return score >= 4
    return score / 4.0

In [127]:
devset, testset = devset[:-10], devset[-10:]

In [129]:
evaluator = Evaluate(devset=devset, metric=tweet_ensemble_metric, num_threads=2, display_progress=True, display_table=5, provide_traceback=True)
evaluator(uncompiled_tweet_writer) 

Average Metric: 38.5 / 40  (96.2): 100%|██████████| 40/40 [01:51<00:00,  2.79s/it] 


,example_tweet,topic,reasoning,pred_tweet,tweet_ensemble_metric
0,"Doing a thing increases your ability to learn about it. In order words, do the thing, then read about it. The experience will give you...",Learning and Experience,"Learning and experience are intertwined; every lesson learned shapes our understanding and influences future decisions. Embracing both successes and failures enriches our journey, fostering growth...","""Learning is a journey, not a destination. Every experience, whether a success or a setback, shapes who we are and how we grow. Let's embrace...",✔️ [1.0]
1,Anyone else ever accidentally get in the shower with a hat still on? https://t.co/t9IOAGcC1h,Everyday Life/Humor,"Everyday life is filled with humorous moments that many can relate to, making it a great topic for a light-hearted tweet. By highlighting a common...","Why do we always open the fridge, stare at the contents, and then close it like we’re expecting a miracle? 🍕🥗 #EverydayLife #Humor",✔️ [1.0]
2,"If you have all the information to make a perfect decision, you missed the opportunity.",Decision Making,"Decision making is a critical skill that impacts every aspect of our lives, from personal choices to professional strategies. It involves evaluating options, considering potential...","🧠💡 Decision making is an essential skill that shapes our lives! Whether in personal choices or professional strategies, making informed decisions can lead to better...",✔️ [1.0]
3,"I’m bad at starting new habits. I’m also pretty bad at keeping habits. In the last ten years, the only small habits I’ve been able...",Habit Formation,Habit formation is a crucial aspect of personal development and productivity. Understanding how habits are formed can help individuals create positive changes in their lives....,"🌱 Want to build better habits? Start small! Consistency is key. Focus on one action at a time, and watch it grow into a lasting...",✔️ [1.0]
4,The reason you are stressed is you have decisions to make and you’re not making them.,Mental Health,"Mental health is a crucial aspect of overall well-being that affects how we think, feel, and act. It's important to raise awareness about mental health...","🧠💚 Let's break the stigma around mental health! It's okay to not be okay. Remember, seeking help is a sign of strength. Let's support each...",✔️ [1.0]


96.25

## Uncompiled: 96.25

In [133]:
config = dict(max_bootstrapped_demos=15, max_labeled_demos=5)
optimizer = BootstrapFewShot(metric = tweet_ensemble_metric, **config)
optimized_tweet_writer = optimizer.compile(TweetWriter(), trainset=trainset[:10])
optimized_tweet_writer.save(path="fewshotv2")

100%|██████████| 10/10 [01:06<00:00,  6.63s/it]

Bootstrapped 9 full traces after 10 examples in round 0.
[('prog', Predict(StringSignature(topic -> reasoning, tweet
    instructions='Given the fields `topic`, produce the fields `tweet`.'
    topic = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Topic:', 'desc': '${topic}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    tweet = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Tweet:', 'desc': '${tweet}'})
)))]


In [134]:
evaluator(optimized_tweet_writer) 

Average Metric: 40.0 / 40  (100.0): 100%|██████████| 40/40 [01:51<00:00,  2.78s/it]


,example_tweet,topic,reasoning,pred_tweet,tweet_ensemble_metric
0,"Doing a thing increases your ability to learn about it. In order words, do the thing, then read about it. The experience will give you...",Learning and Experience,"Learning through experience is one of the most effective ways to gain knowledge and skills. Each experience, whether positive or negative, offers valuable lessons that...",Experience is the best teacher! 📚✨ Every challenge and triumph brings valuable lessons. Embrace your journey and let your experiences shape your growth! #Learning #LifeLessons,✔️ [1.0]
1,Anyone else ever accidentally get in the shower with a hat still on? https://t.co/t9IOAGcC1h,Everyday Life/Humor,"Everyday life can often be mundane, but humor is a powerful tool to lighten our daily experiences. Finding joy and laughter in the little things...",Why did the scarecrow win an award? Because he was outstanding in his field! 🌾😄 Remember to find humor in everyday life—it makes the mundane...,✔️ [1.0]
2,"If you have all the information to make a perfect decision, you missed the opportunity.",Decision Making,"Effective decision-making is crucial in both personal and professional contexts. It involves evaluating options, considering potential outcomes, and aligning choices with our values and goals....","Every decision shapes your path. 🛤️ Take a moment to weigh your options, trust your instincts, and align your choices with your values. Make decisions...",✔️ [1.0]
3,"I’m bad at starting new habits. I’m also pretty bad at keeping habits. In the last ten years, the only small habits I’ve been able...",Habit Formation,Habit formation is a crucial aspect of personal development and achieving long-term goals. Understanding the science behind how habits are formed can help us create...,Small changes lead to big results! 🌱 Focus on forming one positive habit at a time. Consistency is the secret ingredient to lasting change. What...,✔️ [1.0]
4,The reason you are stressed is you have decisions to make and you’re not making them.,Mental Health,"Mental health is a crucial aspect of overall well-being that affects how we think, feel, and act. Prioritizing mental health involves recognizing the importance of...","Mental health matters! 🧠💚 Let’s break the stigma and prioritize our well-being. Remember, it’s okay to ask for help and take time for self-care. You...",✔️ [1.0]


100.0

## BootstrapFewShot: 100

In [135]:
config = dict(max_labeled_demos = 1, max_bootstrapped_demos=1, num_candidate_programs=2)
teleprompter = BootstrapFewShotWithRandomSearch(metric = tweet_ensemble_metric, **config)
rs_optimized_tweet_writer = teleprompter.compile(TweetWriter(), trainset=trainset[:20])
rs_optimized_tweet_writer.save(path="fewshotwithrsv2")

Going to sample between 1 and 1 traces per predictor.
Will attempt to bootstrap 2 candidate sets.


Average Metric: 17.5 / 20  (87.5): 100%|██████████| 20/20 [00:23<00:00,  1.16s/it] 


New best score: 87.5 for seed -3
Scores so far: [87.5]
Best score so far: 87.5


Average Metric: 19.75 / 20  (98.8): 100%|██████████| 20/20 [00:26<00:00,  1.34s/it]


New best score: 98.75 for seed -2
Scores so far: [87.5, 98.75]
Best score so far: 98.75


...

Scores so far: [87.5, 98.75, 100.0, 100.0]
Best score so far: 100.0


  5%|▌         | 1/20 [00:06<02:09,  6.82s/it]


Bootstrapped 1 full traces after 2 examples in round 0.


Average Metric: 20.0 / 20  (100.0): 100%|██████████| 20/20 [00:19<00:00,  1.02it/s]

Scores so far: [87.5, 98.75, 100.0, 100.0, 100.0]
Best score so far: 100.0
5 candidate programs found.
[('prog', Predict(StringSignature(topic -> reasoning, tweet
    instructions='Given the fields `topic`, produce the fields `tweet`.'
    topic = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Topic:', 'desc': '${topic}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    tweet = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Tweet:', 'desc': '${tweet}'})
)))]


In [136]:
evaluator(rs_optimized_tweet_writer) 

Average Metric: 38.0 / 40  (95.0): 100%|██████████| 40/40 [01:39<00:00,  2.49s/it]


,example_tweet,topic,reasoning,pred_tweet,tweet_ensemble_metric
0,"Doing a thing increases your ability to learn about it. In order words, do the thing, then read about it. The experience will give you...",Learning and Experience,"Learning through experience is one of the most effective ways to gain knowledge and skills. It allows individuals to apply theoretical concepts in real-world situations,...","📚✨ Learning through experience is key to personal and professional growth! By applying knowledge in real-world situations, we deepen our understanding and retention. Let's embrace...",✔️ [1.0]
1,Anyone else ever accidentally get in the shower with a hat still on? https://t.co/t9IOAGcC1h,Everyday Life/Humor,"Everyday life is filled with humorous moments that often go unnoticed. Sharing these light-hearted experiences can bring joy and laughter to others, reminding us not...",Why do we always park in driveways and drive on parkways? 🤔 Just another mystery of everyday life! Let's embrace the humor in our daily...,✔️ [1.0]
2,"If you have all the information to make a perfect decision, you missed the opportunity.",Decision Making,"Effective decision making is essential in both personal and professional contexts. It involves analyzing information, weighing options, and considering potential outcomes. Good decision-making skills can...","🧠 Decision making is a vital skill for success! By analyzing information and weighing options, we can make informed choices that lead to better outcomes....",✔️ [1.0]
3,"I’m bad at starting new habits. I’m also pretty bad at keeping habits. In the last ten years, the only small habits I’ve been able...",Habit Formation,"Habit formation is a powerful process that can significantly impact our lives. By understanding the cues, routines, and rewards that drive our behaviors, we can...","🌱 Want to change your life? Start with habit formation! By understanding cues, routines, and rewards, you can build positive habits that lead to personal...",✔️ [1.0]
4,The reason you are stressed is you have decisions to make and you’re not making them.,Mental Health,"Mental health is a vital aspect of overall well-being that affects how we think, feel, and act. Promoting mental health awareness and providing access to...","🧠💚 Mental health matters! Let's break the stigma and promote awareness. It's time to prioritize mental well-being and support those who need it. Remember, it's...",✔️ [1.0]


95.0

## BootstrapFewShotWithRandomSearch: 95

In [137]:
# Initialize optimizer
teleprompter = MIPROv2(
    metric=tweet_ensemble_metric,
    num_candidates=7,
    init_temperature=0.5,
    verbose=False,
    num_threads=2,
)

mipro_optimized_program = teleprompter.compile(
    TweetWriter(),
    trainset=trainset,
    max_bootstrapped_demos=3,
    max_labeled_demos=4,
    num_trials=10,
    minibatch_size=25,
    minibatch_full_eval_steps=10,
    minibatch=True, 
)

# Save optimize program for future use
mipro_optimized_program.save("mipro_optimizedv2")

Projected Language Model (LM) Calls

Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 7 * 1 lm calls in program + (2) lm calls in program aware proposer = 19 prompt model calls
- Task Model: 25 examples in minibatch * 10 batches + 200 examples in val set * 1 full evals = 300 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your

  6%|▌         | 3/50 [00:19<05:11,  6.63s/it]


Bootstrapped 3 full traces after 4 examples in round 0.
Bootstrapping set 4/7


  4%|▍         | 2/50 [00:13<05:28,  6.85s/it]


Bootstrapped 2 full traces after 3 examples in round 0.
Bootstrapping set 5/7


...

Score: 96.0 on minibatch of size 25 with parameters ['Predictor 1: Instruction 1', 'Predictor 1: Few-Shot Set 2'].


== Minibatch Trial 10 / 10 ==


Average Metric: 25.0 / 25  (100.0): 100%|██████████| 25/25 [01:14<00:00,  2.97s/it]


Score: 100.0 on minibatch of size 25 with parameters ['Predictor 1: Instruction 3', 'Predictor 1: Few-Shot Set 2'].


===== Full Eval 1 =====
Doing full eval on next top averaging program (Avg Score: 100.0) so far from mini-batch trials...


Average Metric: 197.0 / 200  (98.5): 100%|██████████| 200/200 [06:58<00:00,  2.09s/it]

Best full eval score so far! Score: 98.5


[('prog', Predict(StringSignature(topic -> reasoning, tweet
    instructions='Given the fields `topic`, produce the fields `tweet`.'
    topic = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Topic:', 'desc': '${topic}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    tweet = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Tweet:', 'desc': '${tweet}'})
)))]


In [138]:
evaluator(mipro_optimized_program)

Average Metric: 40.0 / 40  (100.0): 100%|██████████| 40/40 [01:39<00:00,  2.50s/it]


,example_tweet,topic,reasoning,pred_tweet,tweet_ensemble_metric
0,"Doing a thing increases your ability to learn about it. In order words, do the thing, then read about it. The experience will give you...",Learning and Experience,Learning and experience are intertwined; every lesson learned shapes our journey and equips us for future challenges. Embracing both successes and failures as valuable experiences...,"Every experience, good or bad, is a stepping stone to growth. Embrace the lessons, reflect on your journey, and let each moment shape you into...",✔️ [1.0]
1,Anyone else ever accidentally get in the shower with a hat still on? https://t.co/t9IOAGcC1h,Everyday Life/Humor,"Everyday life can often be mundane, but humor is a powerful tool to lighten our daily experiences. Finding joy and laughter in the little things...",Why do we park in driveways and drive on parkways? 🤔 Life is full of these little mysteries! Embrace the humor in everyday life—it makes...,✔️ [1.0]
2,"If you have all the information to make a perfect decision, you missed the opportunity.",Decision Making,"Decision making is a critical skill that shapes our lives and futures. It involves weighing options, considering consequences, and trusting our instincts. Emphasizing the importance...","Every decision you make is a step on your journey. Trust your instincts, reflect on your values, and remember: it’s okay to take your time....",✔️ [1.0]
3,"I’m bad at starting new habits. I’m also pretty bad at keeping habits. In the last ten years, the only small habits I’ve been able...",Habit Formation,"Habit formation is a powerful process that shapes our daily lives and long-term success. Understanding that habits are built through consistency and small, incremental changes...","Every great achievement starts with a single habit. 🌱 Focus on small, consistent actions, and watch them transform your life over time. Remember, it’s progress,...",✔️ [1.0]
4,The reason you are stressed is you have decisions to make and you’re not making them.,Mental Health,"Mental health is a vital aspect of overall well-being that often gets overlooked. It's important to prioritize self-care, seek help when needed, and foster open...","Your mental health matters just as much as your physical health. 🌱 Take a moment today to check in with yourself, practice self-care, and reach...",✔️ [1.0]


100.0

# MIPROv2 score: 100
Given that the eval for this was more robust, this might be a more 'useful' 100 than a 100 from BootstrapFewShot. I'd like to explore how to normalize and juxtapose these scores but leave it up for further work in the interest of time.

Also during this run, I realised that some of the higher hyperparams in both of my MIPRO runs were completely unneccessary. As such, I'll reduce them for the next experiment.

# Experiment 3: with RAG

In [1]:
import pickle 

with open('data/annotated_dataset.pkl', 'rb') as pickle_file:
    annotated_dataset = pickle.load(pickle_file)

/Users/kevinvegda/Desktop/code/tweet-optim-dspy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rag_entries = 200
#200 entries
set_for_rag = annotated_dataset[:rag_entries]
#100 entries
set_for_training = annotated_dataset[rag_entries:]
#75 for training, 25 for val
dev_set_n = 75
trainset = [x.without('engagement').with_inputs('topic') for x in set_for_training[:dev_set_n]]
devset = [x.without('engagement').with_inputs('topic') for x in set_for_training[dev_set_n:]]

In [3]:
import weaviate
from dotenv import load_dotenv
import os

load_dotenv()
  
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = weaviate.connect_to_local(host='localhost', port=8080, headers={
    'X-Openai-Api-Key': OPENAI_API_KEY
})

print(client.is_ready())

True


In [4]:
import weaviate.classes.config as wvcc
  
client.collections.delete_all()
collection = client.collections.create(
    name = "DspyTweets",
    vectorizer_config=wvcc.Configure.Vectorizer.text2vec_openai(),
    properties=[
        wvcc.Property(name = "content", data_type=wvcc.DataType.TEXT),
    ],
)

In [5]:
import re

def chunk_list(lst, chunk_size):
    """Break a list into chunks of the specified size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

def split_into_sentences(text):
    """Split text into sentences using regular expressions."""
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

def chunk_tweets():
    """Read index.md files from subfolders, split into sentences, and chunk every 5 sentences."""
    tweet_chunks = []
    for tweet in set_for_rag:
        content = tweet.tweet
        sentences = split_into_sentences(content)
        sentence_chunks = chunk_list(sentences, 5)
        sentence_chunks = [' '.join(chuck) for chuck in sentence_chunks]
        tweet_chunks.extend(sentence_chunks)
    return tweet_chunks

tweet_chunks = chunk_tweets()

In [6]:
len(tweet_chunks)

205

In [7]:
tweet_chunks[0]

'Stop “friend financing”'

In [8]:
tweets_collection = client.collections.get("DspyTweets")

for idx, tweet_chunk in enumerate(tweet_chunks):
    upload = tweets_collection.data.insert(
        properties={
            "content": tweet_chunk
        }
    )

In [9]:
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM

retriever_model = WeaviateRM(
    weaviate_collection_name="DspyTweets",
    weaviate_client=client,
)

results = retriever_model("entrepreneurship", k=5)

for result in results:
    print("Document:", result.long_text, "\n")

Document: Entrepreneurship is less about being really good at anything and more about being good enough at everything. 

Document: Entrepreneurship isn’t about being great at a few things but being good enough at everything. 

Document: The best entrepreneurs don’t do business to make money. They make money to do business. 

Document: You can always justify doing a new thing by quantifying what you have to gain. The tough part is quantifying what you’ll lose from the distraction. And that downside is the part most entrepreneurs miscalculate. 

Document: Many small business owners stay small because they don’t have the patience to let something small become big. 



In [10]:
dspy.configure(rm = retriever_model)

In [11]:
# Define the signature for automatic assessments.
gpt4o = dspy.LM(model= 'gpt-4o', max_tokens=1000, model_type='chat')
retrieve = dspy.Retrieve(k=5)

class Assess(dspy.Signature):
    """Assess the quality of a tweet along the specified dimension."""

    context = dspy.InputField(desc='ignore if N/A')
    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="Yes or No")

def metric(gold, pred, trace=None):
    topic, tweet = gold.topic, pred.tweet
    context = retrieve(topic).passages

    engaging = "Does the assessed text make for a self-contained, engaging tweet?"
    faithful = "Is the assessed text grounded in the context? Say no if it includes significant facts not in the context."
    concise = "Does the assessed text make for a concise, cogent tweet?"
    creative = "Does the assessed text make for a creative tweet?"
    relevant = f"The this is the tweet: `{tweet}`. It should be relevant to: `{topic}`."
    
    with dspy.context(lm=gpt4o):
        faithful = dspy.Predict(Assess)(context=context, assessed_text=tweet, assessment_question=faithful)
        relevant =  dspy.Predict(Assess)(context='N/A', assessed_text=tweet, assessment_question=relevant)
        engaging = dspy.Predict(Assess)(context='N/A', assessed_text=tweet, assessment_question=engaging)
        creative = dspy.Predict(Assess)(context='N/A', assessed_text=tweet, assessment_question=creative)
        concise = dspy.Predict(Assess)(context='N/A', assessed_text=tweet, assessment_question=concise)

    relevant, engaging, creative, concise, faithful = (m.assessment_answer.split()[0].lower() == 'yes' for m in [relevant, engaging, creative, concise, faithful])
    #takes care of length
    score = (relevant + engaging + creative + concise + faithful) if (len(tweet) <= 280) else 0

    if trace is not None: 
        return score >= 5
    return score / 5.0

In [12]:
class GenerateTweet(dspy.Signature):
    """Generate a tweet based on the context"""

    context = dspy.InputField(desc = "May contain relevant facts")
    topic = dspy.InputField()
    tweet = dspy.OutputField()

class TweetRAG(dspy.Module):
    def __init__(self, num_passages = 3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k = num_passages)
        self.generate_tweet = dspy.ChainOfThought(GenerateTweet)

    def forward(self, topic):
        context = self.retrieve(topic).passages
        prediction = self.generate_tweet(context = context, topic = topic)
        return dspy.Prediction(tweet = prediction.tweet)

In [13]:
mini = dspy.LM(model = 'gpt-4o-mini')
dspy.configure(lm=mini)

In [14]:
from dspy.evaluate.evaluate import Evaluate
uncompiled_tweet_rag = TweetRAG()
evaluator = Evaluate(devset=devset, metric=metric, num_threads=2, display_progress=True, display_table=5, provide_traceback=True)
evaluator(uncompiled_tweet_rag) 

Average Metric: 22.8 / 25  (91.2): 100%|██████████| 25/25 [01:08<00:00,  2.75s/it]


,example_tweet,topic,pred_tweet,metric
0,For those who ask how I eat desert every night and don’t get fat… Answer: I eat less during the day.,Diet and Weight Management,"""Weight management isn't about winning; it's about the journey. Just like in marriage, the goal is to keep going. Celebrate the small victories in the...",✔️ [1.0]
1,You can give up tomorrow. Today you fight.,Motivation,"Motivation doesn't always come from passion. Sometimes, it's pain, fear, or even shame that drives us. Remember, every small win counts! Visualize your audience—93 views?...",
2,There are no recessions. Only buying seasons and selling seasons. Your skills determine which season it is for you.,Economic Perspective,"💡 Everyone is an investor! Whether rich or poor, how we trade our resources defines our returns. Start measuring your investments in time and money...",✔️ [1.0]
3,A life tip that’s served me well: Save your big decisions for a morning after a good nights sleep.,Life Advice,"Life isn't a race to win; it's a journey to enjoy. Focus on nurturing relationships, health, and passions rather than just reaching a destination. Sometimes,...",✔️ [1.0]
4,Whenever I get to a low point where I think “why do I even bother?” I just try to remind myself “this is where most...,Motivation and Perseverance,"""Don't wait for motivation to strike! Start when it’s hard and use your challenges as fuel. Pain, fear, and anger can be powerful motivators. Conquer...",✔️ [1.0]


91.2

## Uncompiled Tweet-RAG: 91.2

In [15]:
from dspy.teleprompt import BootstrapFewShot
config = dict(max_bootstrapped_demos=15, max_labeled_demos=5)
optimizer = BootstrapFewShot(metric = metric, **config)
optimized_tweet_writer = optimizer.compile(TweetRAG(), trainset=trainset[:10])
optimized_tweet_writer.save(path="fewshotv3")

100%|██████████| 10/10 [01:03<00:00,  6.33s/it]

Bootstrapped 8 full traces after 10 examples in round 0.
[('retrieve', <dspy.retrieve.retrieve.Retrieve object at 0x132e64890>), ('generate_tweet', Predict(StringSignature(context, topic -> reasoning, tweet
    instructions='Generate a tweet based on the context'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'May contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    topic = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Topic:', 'desc': '${topic}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    tweet = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Tweet:', 'desc': '${tweet}'})
)))]


In [16]:
evaluator(optimized_tweet_writer)

Average Metric: 24.8 / 25  (99.2): 100%|██████████| 25/25 [01:21<00:00,  3.24s/it] 


,example_tweet,topic,pred_tweet,metric
0,For those who ask how I eat desert every night and don’t get fat… Answer: I eat less during the day.,Diet and Weight Management,"In diet and weight management, there’s no “winning”—only ongoing commitment. Focus on mastering the middle, where real progress happens. Celebrate the journey, not just the...",✔️ [1.0]
1,You can give up tomorrow. Today you fight.,Motivation,"Motivation can come from unexpected places—pain, fear, or even shame. Use what you’ve got! And remember, every small win matters. Imagine those views as a...",✔️ [1.0]
2,There are no recessions. Only buying seasons and selling seasons. Your skills determine which season it is for you.,Economic Perspective,"In the economy of life, we’re all investors trading resources for returns. Recognize your time as your most valuable asset! The rarer your perspective, the...",✔️ [1.0]
3,A life tip that’s served me well: Save your big decisions for a morning after a good nights sleep.,Life Advice,"Life isn’t about winning; it’s about playing the game. Whether in marriage, health, or business, focus on the journey, not just the destination. Sometimes, the...",✔️ [1.0]
4,Whenever I get to a low point where I think “why do I even bother?” I just try to remind myself “this is where most...,Motivation and Perseverance,Don’t wait for the perfect moment to start—begin when it’s hard! 💪 Conquering tiny impulses builds the perseverance needed for massive dreams. Use whatever motivates...,✔️ [1.0]


99.2

## BootstrapFewShot: 99.2

In [17]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
config = dict(max_labeled_demos = 2, max_bootstrapped_demos=2, num_candidate_programs=3)
teleprompter = BootstrapFewShotWithRandomSearch(metric = metric, **config)
rs_optimized_tweet_writer = teleprompter.compile(TweetRAG(), trainset=trainset[:20])
rs_optimized_tweet_writer.save(path="fewshotwithrsv3")

Going to sample between 1 and 2 traces per predictor.
Will attempt to bootstrap 3 candidate sets.


Average Metric: 17.2 / 20  (86.0): 100%|██████████| 20/20 [00:25<00:00,  1.26s/it]              


New best score: 86.0 for seed -3
Scores so far: [86.0]
Best score so far: 86.0


Average Metric: 18.6 / 20  (93.0): 100%|██████████| 20/20 [00:22<00:00,  1.11s/it]


New best score: 93.0 for seed -2
Scores so far: [86.0, 93.0]
Best score so far: 93.0


...

Scores so far: [86.0, 93.0, 94.0, 99.0, 70.0]
Best score so far: 99.0


  5%|▌         | 1/20 [00:07<02:19,  7.33s/it]


Bootstrapped 1 full traces after 2 examples in round 0.


Average Metric: 19.8 / 20  (99.0): 100%|██████████| 20/20 [00:24<00:00,  1.25s/it]

Scores so far: [86.0, 93.0, 94.0, 99.0, 70.0, 99.0]
Best score so far: 99.0
6 candidate programs found.
[('retrieve', <dspy.retrieve.retrieve.Retrieve object at 0x134570380>), ('generate_tweet', Predict(StringSignature(context, topic -> reasoning, tweet
    instructions='Generate a tweet based on the context'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'May contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    topic = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Topic:', 'desc': '${topic}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    tweet = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Tweet:', 'desc': '${tweet}'})
)))]


In [18]:
mini.inspect_history(n=1)





System message:

Your input fields are:
1. `context` (str): May contain relevant facts
2. `topic` (str)

Your output fields are:
1. `reasoning` (str)
2. `tweet` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## topic ## ]]
{topic}

[[ ## reasoning ## ]]
{reasoning}

[[ ## tweet ## ]]
{tweet}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Generate a tweet based on the context


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## context ## ]]
Not supplied for this particular example.

[[ ## topic ## ]]
Workforce Management

Respond with the corresponding output fields, starting with the field `reasoning`, then `tweet`, and then ending with the marker for `completed`.


Assistant message:

[[ ## reasoning ## ]]
Not supplied for this particular example.

[[ ## tweet ## ]]
Raising salaries can attract bett

In [19]:
evaluator(rs_optimized_tweet_writer)

Average Metric: 24.8 / 25  (99.2): 100%|██████████| 25/25 [01:08<00:00,  2.74s/it] 


,example_tweet,topic,pred_tweet,metric
0,For those who ask how I eat desert every night and don’t get fat… Answer: I eat less during the day.,Diet and Weight Management,"Diet and weight management aren’t about “winning” but about playing the long game. Focus on consistency and self-motivation, especially in the middle of your journey....",✔️ [1.0]
1,You can give up tomorrow. Today you fight.,Motivation,"Motivation doesn't always come from passion; sometimes it’s pain, fear, or anger that drives us. Embrace what you have! Visualize your small wins as a...",✔️ [1.0]
2,There are no recessions. Only buying seasons and selling seasons. Your skills determine which season it is for you.,Economic Perspective,"Everyone is an investor in their own life! How you spend your time reflects your priorities and potential returns. As you rise, remember: the rarer...",✔️ [1.0]
3,A life tip that’s served me well: Save your big decisions for a morning after a good nights sleep.,Life Advice,"Life's best games can't be won, only played. Focus on the journey, not just the destination. Sometimes, giving up the sure thing leads to the...",✔️ [1.0]
4,Whenever I get to a low point where I think “why do I even bother?” I just try to remind myself “this is where most...,Motivation and Perseverance,"Start your journey when it’s hard! Conquering tiny impulses builds the perseverance needed for massive dreams. Use what you’ve got—pain, fear, or anger can be...",✔️ [1.0]


99.2

## BootstrapFewShotWithRandomSearch: 99.2

In [20]:
from dspy.teleprompt import MIPROv2
# Initialize optimizer
teleprompter = MIPROv2(
    metric=metric,
    num_candidates=7,
    init_temperature=0.5,
    verbose=False,
    num_threads=2,
)

mipro_optimized_program = teleprompter.compile(
    TweetRAG(),
    trainset=trainset,
    max_bootstrapped_demos=3,
    max_labeled_demos=4,
    num_trials=5,
    minibatch=True,
    minibatch_size=5, 
    minibatch_full_eval_steps=2
)

# Save optimize program for future use
mipro_optimized_program.save("mipro_optimizedv3")

Projected Language Model (LM) Calls

Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 7 * 1 lm calls in program + (2) lm calls in program aware proposer = 19 prompt model calls
- Task Model: 5 examples in minibatch * 5 batches + 60 examples in val set * 2 full evals = 55 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your bud

 27%|██▋       | 4/15 [00:27<01:14,  6.76s/it]


Bootstrapped 3 full traces after 5 examples in round 0.
Bootstrapping set 4/7


 20%|██        | 3/15 [00:19<01:18,  6.55s/it]


Bootstrapped 2 full traces after 4 examples in round 0.
Bootstrapping set 5/7


...

Full eval score: 69.0
Best full eval score so far: 92.33


== Minibatch Trial 5 / 5 ==


Average Metric: 3.0 / 5  (60.0): 100%|██████████| 5/5 [00:23<00:00,  4.62s/it]


Score: 60.0 on minibatch of size 5 with parameters ['Predictor 1: Instruction 6', 'Predictor 1: Few-Shot Set 1'].


===== Full Eval 3 =====
Doing full eval on next top averaging program (Avg Score: 92.0) so far from mini-batch trials...


Average Metric: 49.99999999999999 / 60  (83.3): 100%|██████████| 60/60 [02:59<00:00,  2.98s/it] 

Full eval score: 83.33
Best full eval score so far: 92.33


[('retrieve', <dspy.retrieve.retrieve.Retrieve object at 0x1351d65d0>), ('generate_tweet', Predict(StringSignature(context, topic -> reasoning, tweet
    instructions='Generate a tweet based on the context'
    context = Field(annotation=str required=True json_schema_extra={'desc': 'May contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    topic = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Topic:', 'desc': '${topic}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    tweet = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Tweet:', 'desc': '${tweet}'})
)))]


In [22]:
mini.inspect_history(n=1)





System message:

Your input fields are:
1. `context` (str): May contain relevant facts
2. `topic` (str)

Your output fields are:
1. `reasoning` (str)
2. `tweet` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## topic ## ]]
{topic}

[[ ## reasoning ## ]]
{reasoning}

[[ ## tweet ## ]]
{tweet}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        In a competitive landscape where every tweet can influence public perception and engagement, generate a compelling tweet that encapsulates the provided context and topic. Ensure that the reasoning clearly articulates the insights drawn from the context, highlighting the significance of the topic in a way that resonates with the audience and encourages interaction.


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## context ## ]]
Not supplied for this particular examp

In [23]:
evaluator(mipro_optimized_program)

Average Metric: 22.8 / 25  (91.2): 100%|██████████| 25/25 [00:12<00:00,  2.04it/s]


,example_tweet,topic,pred_tweet,metric
0,For those who ask how I eat desert every night and don’t get fat… Answer: I eat less during the day.,Diet and Weight Management,"""Weight management isn't about winning; it's about the journey. Just like in marriage, the goal is to keep going. Celebrate the small victories in the...",✔️ [1.0]
1,You can give up tomorrow. Today you fight.,Motivation,"Motivation doesn't always come from passion. Sometimes, it's pain, fear, or even shame that drives us. Remember, every small win counts! Visualize your audience—93 views?...",
2,There are no recessions. Only buying seasons and selling seasons. Your skills determine which season it is for you.,Economic Perspective,"💡 Everyone is an investor! Whether rich or poor, how we trade our resources defines our returns. Start measuring your investments in time and money...",✔️ [1.0]
3,A life tip that’s served me well: Save your big decisions for a morning after a good nights sleep.,Life Advice,"Life isn't a race to win; it's a journey to enjoy. Focus on nurturing relationships, health, and passions rather than just reaching a destination. Sometimes,...",✔️ [1.0]
4,Whenever I get to a low point where I think “why do I even bother?” I just try to remind myself “this is where most...,Motivation and Perseverance,"""Don't wait for motivation to strike! Start when it’s hard and use your challenges as fuel. Pain, fear, and anger can be powerful motivators. Conquer...",✔️ [1.0]


91.2

## MIPROv2 with RAG: 91.2

# Experiment 4: RAG evaluation with RAGAS
So far we've only used the custom metrics we defined with the LLM judge, but we could integrate some objective metrics defined by RAGAS into our evaluation/training. Let's try that.